In [ ]:
import os
import shutil
os.chdir('C:/Users/Carlo/')

In this section, we'll see how to transform a 3D file so that it can be read by our Python program.

Let’s begin.

Our bounding boxes are the volumes where, when the hands enter, we can say the operator is working on a component X.

To define these volumes, you can either:

1-Manually write a .yml file specifying the coordinates of the nearest and farthest vertices of the box (relative to the origin), structured like the file NOTEBOOKS/boxes_example.yml.

2-Alternatively, use a software called FreeCAD to define the volumes, save them in STEP format, generate meshes, and then convert them into .yml files via a terminal command (you can find this command in NOTEBOOKS/STEP_TO_YML.py).

If the boxes folder doesn’t exist in the data directory, create it using the next step. Then, place the newly created boxes.yml file inside it.
Important! The file must be named boxes.yml!

To verify the correct structure, you can use the viewboxes function...

In [ ]:
import os
import shutil
os.makedirs('EasyMocap/data/boxes', exist_ok=True)
shutil.copy('EasyMocap/yamls/REORIENTED/extri.yml','EasyMocap/data/boxes/')
shutil.copy('EasyMocap/yamls/REORIENTED/intri.yml','EasyMocap/data/boxes/')

In [ ]:
!python EasyMocap/apps/hopeitworks/viewboxes.py EasyMocap/data/boxes/boxes.yml

To project the boxes into our real-world system (video frames) to verify if the bounding boxes align with reality, we can use the following code cell. The results can be viewed in: EasyMocap/data/boxes/boxes/

In [ ]:
!python EasyMocap/apps/hopeitworks/boxesprojection.py EasyMocap/data --out EasyMocap/data/boxes/ --boxes-yml EasyMocap/data/boxes/boxes.yml --write

Now, with the next cell, we’ll clean the previously used 'data' folder (employed for motion capture of keypoints to define O-X-Y, if populated) and proceed with:

1-Loading new videos into data/videos.

2-Detecting keypoints as shown in Tutorial 3...

These videos capture the operator performing activities on the various components of the product in question.

In [ ]:
import shutil
import os

!python EasyMocap/apps/hopeitworks/cleanhandmocap.py
os.remove('EasyMocap/data/extri.yml')  
os.remove('EasyMocap/data/intri.yml')  
shutil.copy('EasyMocap/yamls/REORIENTED/extri.yml','EasyMocap/data/')
shutil.copy('EasyMocap/yamls/REORIENTED/intri.yml','EasyMocap/data/')

With the next cell (warning: it will take a while), we repeat the steps already seen in Tutorial 3: images → to annots → to kp3d.

In [ ]:
##ESTRAE IMMAGINI DA VIDEO
import time
import os
# Inizio del timer
start_time = time.time()

# ESTRAZIONE IMMAGINI
!python EasyMocap/apps/preprocess/extract_image.py EasyMocap/data

# Fine del timer
end_time = time.time()
elapsed_time = end_time - start_time
# Salva il tempo in un file txt
with open('EasyMocap/data/times.txt', 'a') as f:
    f.write(f't extract_image, {elapsed_time:.2f}\n')
##ESTRAE KEYPOINTS
import time
import os
import json
import shutil
# Inizio del timer
start_time = time.time()

## MANO SINISTRA
!python EasyMocap/apps/preprocess/extract_keypoints.py EasyMocap/data --mode mp-handl
## RINOMINA ANNOTS IN ANNOTSL PER CONVENIENZA DEL SUCCESSIVO ANNOT (DELLA MANO DESTRA)
os.rename('EasyMocap/data/annots', 'EasyMocap/data/annotsl')
## MANO DESTRA
!python EasyMocap/apps/preprocess/extract_keypoints.py EasyMocap/data --mode mp-handr


##INIZIO JOIN DELLE DUE CARTELLE ANNOTS

# Define the path to your main folder
main_folder_path = 'EasyMocap/data/annotsl'

# Get a list of sub-folders within the main folder
subfolders = [f.path for f in os.scandir(main_folder_path)]

# Loop through each sub-folder
for subfolder_path in subfolders:

    # Get a list of JSON files within the sub-folder
    json_files = [f.path for f in os.scandir(subfolder_path)]

    # Loop through the JSON files in the sub-folder
    for json_file_path in json_files:

        # You can now read and process the JSON file
        with open(json_file_path, 'r') as json_file:
            data = json.load(json_file)
            json_file_path_dest = json_file_path.replace("annotsl", "annots")

            # Step 1: Read the existing data from the destination JSON file
            with open(json_file_path_dest, 'r') as json_file:
                existing_data = json.load(json_file)

            # Step 2: Modify or add new data to the existing data
            existing_data["annots"][0]["handl2d"] =  data["annots"][0]["handl2d"]
            existing_data["annots"][0]["bbox_handl2d"] =  data["annots"][0]["bbox_handl2d"]

            # Step 3: Write the updated data back to the JSON file
            with open(json_file_path_dest, 'w') as json_file:
                json.dump(existing_data, json_file, indent=4)

shutil.rmtree('EasyMocap/data/annotsl')
##FINE JOIN DELLE DUE CARTELLE ANNOTS

# Fine del timer
end_time = time.time()
elapsed_time = end_time - start_time
# Salva il tempo in un file txt
with open('EasyMocap/data/times.txt', 'a') as f:
    f.write(f't extract_keypoints, {elapsed_time:.2f}\n')

##in times.txt, SCRIVERE A INIZIO FILE NUMERO FRAME E NUMERO FPS CON QUESTO FORMATO EX. (#FRAMES, 434 /n #FPS, 9)
##TRIANGOLAZIONE K2D->K3D
import time

# Inizio del timer
start_time = time.time()

!python EasyMocap/apps/fit/triangulate1p.py --cfg_data EasyMocap/config/recon/mv1p.yml --opt_data args.path EasyMocap/data args.out EasyMocap/data/output-keypoints3d --cfg_exp EasyMocap/config/recon/triangulator-mv1p-total.yml --opt_exp  args.debug True

# Fine del timer
end_time = time.time()
elapsed_time = end_time - start_time
# Salva il tempo in un file txt
with open('EasyMocap/data/times.txt', 'a') as f:
    f.write(f't triangulate, {elapsed_time:.2f}\n')

**ITERATIVE STEP** 


Now we proceed with detecting whether the keypoints and bounding boxes come into contact and intersect using the following function:

P.S. If this isn't your first time here (you've already completed the steps below), activate and run cleanboxes, where we'll clean the 'boxes' folder from the data.

In [ ]:
#!python EasyMocap/apps/hopeitworks/cleanboxes.py 

!python EasyMocap/apps/hopeitworks/rilevak3d.py EasyMocap/data/boxes/boxes.yml EasyMocap/data/output-keypoints3d/keypoints3d EasyMocap/data/boxes/results.csv

And with the creation of the images (this time with the boxes changing color):

In [ ]:
!python EasyMocap/apps/hopeitworks/boxesprojection.py EasyMocap/data --out EasyMocap/data/boxes/ --boxes-yml EasyMocap/data/boxes/boxes.yml --results EasyMocap/data/boxes/results.csv --write

The following cell will allow us to create a video where we can see our results.
WARNING!! REPLACE CAM1, CAM2, CAM3, ETC. with the names of the cameras (from the original videos).

You will find the videos in the path → EasyMocap/data/boxes/boxes.

In [ ]:
##CREA VIDEO
import cv2
video = cv2.VideoCapture("EasyMocap/data/videos/CAM1.mp4");
fps = video.get(cv2.CAP_PROP_FPS)
print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
fps = int(fps)

!python -m easymocap.visualize.ffmpeg_wrapper EasyMocap/data/boxes/boxes/CAM1 --fps $fps
!python -m easymocap.visualize.ffmpeg_wrapper EasyMocap/data/boxes/boxes/CAM2 --fps $fps
!python -m easymocap.visualize.ffmpeg_wrapper EasyMocap/data/boxes/boxes/CAM3 --fps $fps
!python -m easymocap.visualize.ffmpeg_wrapper EasyMocap/data/boxes/boxes/CAM4 --fps $fps


Once the videos are analyzed, you can choose whether to be satisfied with the results or modify the geometries of the bounding boxes using the tool provided below (TO BE RUN IN THE TERMINAL!) and repeat the process starting again from the ITERATIVE STEP.

In [ ]:
#python EasyMocap/apps/hopeitworks/boxestool.py EasyMocap/data/boxes/boxes.yml

AU REVOIR!